In [4]:
raw_text = '''
Dorothy watched the rhino. 
It sniffed the pumpkin. She sneezed. 
The rhino raised its head. Snorted. Dorothy waved. 
The rhino pawed the earth. She threw a high heel. It hit the rhino. 
The rhino ate the shoe. She yelled, “Hey!” Stomped her foot. Insecure? 
Don’t worry; you’re not alone. However, stay that way and you’ll never accomplish anything of significance. 
Fortunately, there’s a book called Insecure No
More, which will teach you how to be confident and courageous in just 30 days. Buy it now.
'''
    
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
raw_text_split = raw_text.lower().split('.')
tokenizer.fit_on_texts(raw_text_split)

sequential = []
for sent in raw_text_split:
    non_sequential = tokenizer.texts_to_sequences([sent])[0]
    
    if non_sequential:
        for i in range(1, len(non_sequential)):
            sequential.append(non_sequential[:(i+1)])
            
from keras.preprocessing.sequence import pad_sequences
max_len_sentence = max([len(i) for i in sequential])
uniform_data = pad_sequences(sequential, maxlen=max_len_sentence)

vocab_size = len(tokenizer.word_index) + 1
vocab_size, uniform_data

(68, array([[ 0,  0,  0, ...,  0,  5,  9],
        [ 0,  0,  0, ...,  5,  9,  1],
        [ 0,  0,  0, ...,  9,  1,  2],
        ...,
        [47, 48,  6, ..., 63, 64, 65],
        [ 0,  0,  0, ...,  0, 66,  3],
        [ 0,  0,  0, ..., 66,  3, 67]], dtype=int32))

In [47]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM, Dropout
from keras.callbacks import TensorBoard
import time

model = Sequential()

NAME_MODEL = f'sentence-formation=>{time.time()}'

model.add(Embedding(vocab_size, 10))

model.add(LSTM(128, return_sequences=True, activation='relu'))
#model.add(Dropout(0.2))

model.add(LSTM(64))

model.add(Dense(vocab_size, activation='softmax'))

tensorboard = TensorBoard('./logs/{}'.format(NAME_MODEL))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
             metrics=['accuracy'])

In [48]:
x_train, y_train = uniform_data[:, :-1], uniform_data[:, -1]

In [49]:
model.fit(x_train, y_train, callbacks=[tensorboard], epochs=120, verbose=3)

Epoch 1/120
Epoch 2/120
Epoch 3/120
Epoch 4/120
Epoch 5/120
Epoch 6/120
Epoch 7/120
Epoch 8/120
Epoch 9/120
Epoch 10/120
Epoch 11/120
Epoch 12/120
Epoch 13/120
Epoch 14/120
Epoch 15/120
Epoch 16/120
Epoch 17/120
Epoch 18/120
Epoch 19/120
Epoch 20/120
Epoch 21/120
Epoch 22/120
Epoch 23/120
Epoch 24/120
Epoch 25/120
Epoch 26/120
Epoch 27/120
Epoch 28/120
Epoch 29/120
Epoch 30/120
Epoch 31/120
Epoch 32/120
Epoch 33/120
Epoch 34/120
Epoch 35/120
Epoch 36/120
Epoch 37/120
Epoch 38/120
Epoch 39/120
Epoch 40/120
Epoch 41/120
Epoch 42/120
Epoch 43/120
Epoch 44/120
Epoch 45/120
Epoch 46/120
Epoch 47/120
Epoch 48/120
Epoch 49/120
Epoch 50/120
Epoch 51/120
Epoch 52/120
Epoch 53/120
Epoch 54/120
Epoch 55/120
Epoch 56/120
Epoch 57/120
Epoch 58/120
Epoch 59/120
Epoch 60/120
Epoch 61/120
Epoch 62/120
Epoch 63/120
Epoch 64/120
Epoch 65/120
Epoch 66/120
Epoch 67/120
Epoch 68/120
Epoch 69/120
Epoch 70/120
Epoch 71/120
Epoch 72/120
Epoch 73/120
Epoch 74/120
Epoch 75/120
Epoch 76/120
Epoch 77/120
Epoch 78

In [46]:
model.evaluate(x_train, y_train)

71/71 [==============================] - 1s 12ms/step


[3.1741566557279777, 0.09859154929577464]

In [50]:
def generate_text(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [51]:
generate_text('The rhino', 3)

'The rhino the the the'